In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings; warnings.simplefilter('ignore', FutureWarning)
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import pymongo
from pymongo import MongoClient
from avian_vocalizations.evaluator import ParamSpace, EvaluatorFactory
from hyperopt.mongoexp import MongoTrials, MongoCtrl, MongoJobs, MongoWorker
from hyperopt import hp, fmin, tpe
from vastai.api import VastClient
from toolbench.tools import print_module_versions
print_module_versions(globals())

Using TensorFlow backend.


             IPython  {'7.9.0'}
          tensorflow  {'1.15.0'}
             pymongo  {'3.9.0'}
 avian_vocalizations  {'0.0.1'}
            hyperopt  {'0.2.2'}
           toolbench  {'0.0.1'}


In [3]:
vast = VastClient().authenticate()

Initializing vast.ai client with api_key from /home/sam/.vast_api_key.


In [4]:
vast.get_instances()

[405752: exited    stopped              $0.0181/hr ssh5.vast.ai:15752  166.2↑ 229.5↓  flops:2.46144T
 	1X 2.0GB GTX 1050, 32.0/32X 31.6GB Xeon® Silver 4109T 
 	Successfully loaded tensorflow/tensorflow:1.15.0-gpu-py3-jupyter,
 405814: exited    stopped              $0.0181/hr ssh4.vast.ai:15814  166.2↑ 229.5↓  flops:2.46144T
 	1X 2.0GB GTX 1050, 32.0/32X 31.6GB Xeon® Silver 4109T 
 	Successfully loaded tensorflow/tensorflow:1.15.0-gpu-py3-jupyter,
 405882: running   running              $0.1031/hr ssh5.vast.ai:15882  259.7↑ 345.0↓  flops:14.056448T
 	1X 10.9GB GTX 1080 Ti, 5.0/40X 63.7GB Xeon® E5-2680 v2 
 	Successfully loaded tensorflow/tensorflow:1.15.0-gpu-py3-jupyter]

In [5]:
instance = vast.get_running_instances()[0]
# instance.start()
remote = instance.pb_remote

In [6]:
instance.ssh_connection_command

'ssh root@ssh5.vast.ai -p 15882 -i /home/sam/.ssh/vastai'

In [ ]:
# instance.destroy()

## Test tunnel to mongodb

In [6]:
print(remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

2019-11-05T04:51:28.728+0000 I NETWORK  [listener] connection accepted from 127.0.0.1:44996 #2 (2 connections now open)
2019-11-05T04:51:28.729+0000 I NETWORK  [conn2] received client metadata from 127.0.0.1:44996 conn: { driver: { name: "PyMongo", version: "3.9.0" }, os: { type: "Linux", name: "Linux", architecture: "x86_64", version: "4.15.0-45-generic" }, platform: "CPython 3.6.8.final.0" }



In [9]:
with instance.get_tunnel(27018,27017) as tunn:
    mongo = MongoClient('localhost', 27018)
    print(mongo.list_database_names())

['admin', 'config', 'local']


## Hot update evaluator code

In [17]:
remote.sftp.put('avian_vocalizations/evaluator.py',
                '/root/avian_vocalizations/avian_vocalizations/evaluator.py')

<SFTPAttributes: [ size=5768 uid=0 gid=0 mode=0o100644 atime=1572920912 mtime=1572929956 ]>

In [68]:
remote.sftp.put('avian_vocalizations/data.py',
                '/root/avian_vocalizations/avian_vocalizations/data.py')

<SFTPAttributes: [ size=13787 uid=0 gid=0 mode=0o100644 atime=1572122143 mtime=1572122578 ]>

In [ ]:
remote.sftp.put('avian_vocalizations/model.py',
                '/root/avian_vocalizations/avian_vocalizations/model.py')

## Update code repo

In [12]:
with remote.cwd('/root/avian_vocalizations'):
    print(remote['git']('pull'))
    print(remote['git']('status'))

Already up to date.

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	logfile.txt
	start_hyperopt_worker.sh
	xenocanto-avian-vocalizations-canv-usa.zip8lq7wo8g.tmp

nothing added to commit but untracked files present (use "git add" to track)



## Run some trials

In [7]:
tunnel = instance.get_tunnel(27018, 27017)

In [12]:
trials = MongoTrials('mongo://localhost:27018/avian_vocalizations/jobs', exp_key='alpha0')
trials.delete_all()
hp_space = ParamSpace(n_frames=128, 
                      dropout_rate=hp.uniform('dropout',0,.5),
                      batch_size=hp.choice('batch_size',[64]),
                     )

evaluator = EvaluatorFactory(n_splits=2, n_epochs=100, data_dir='data')

best = fmin(evaluator, hp_space, trials=trials, algo=tpe.suggest, max_evals=1)
best

100%|██████████| 1/1 [09:26<00:00, 566.19s/trial, best loss: 3.4742796421051025]


{'batch_size': 0, 'dropout': 0.11778094436205577}

In [13]:
trials.best_trial

SON([('_id', ObjectId('5dc1f9e897931d057d87f3a7')), ('state', 2), ('tid', 64), ('spec', None), ('result', SON([('status', 'ok'), ('loss', 3.4742796421051025), ('loss_variance', 0.0011314028006950139), ('accuracy', 0.1474609375), ('accuracy_variance', 0.00011539459228515625), ('scores', [SON([('loss', 3.440643310546875), ('accuracy', 0.158203125), ('argmin_loss', 48)]), SON([('loss', 3.50791597366333), ('accuracy', 0.13671875), ('argmin_loss', 43)])]), ('trial_time_s', 559)])), ('misc', SON([('tid', 64), ('cmd', ['domain_attachment', 'FMinIter_Domain']), ('workdir', None), ('idxs', SON([('batch_size', [64]), ('dropout', [64])])), ('vals', SON([('batch_size', [0]), ('dropout', [0.11778094436205577])]))])), ('exp_key', 'alpha0'), ('owner', ['bb98ff060dc8:15943']), ('version', 605), ('book_time', datetime.datetime(2019, 11, 5, 22, 38, 32, 984000)), ('refresh_time', datetime.datetime(2019, 11, 5, 22, 47, 56, 243000)), ('_attachments', [['params', ObjectId('5dc1f9ec177b4d0c93f2d705')], ['met

In [14]:
trials.trial_attachments(trials.best_trial).items()

[('params',
  b'{"n_frames": 128, "dropout_rate": 0.11778094436205577, "batch_size": 64}'),
 ('metrics.split0',
  b'{"val_loss": [4.511808395385742, 4.510502815246582, 4.494348049163818, 4.486605644226074, 4.499076843261719, 4.376770973205566, 4.356110572814941, 4.444511413574219, 4.272552490234375, 4.239313125610352, 4.211963176727295, 4.092220306396484, 3.947756767272949, 3.8693645000457764, 3.9441442489624023, 4.018782138824463, 3.7871460914611816, 3.7919230461120605, 3.918069362640381, 3.9134621620178223, 3.685044527053833, 4.1232709884643555, 3.9726336002349854, 3.6652421951293945, 3.7791686058044434, 3.72754168510437, 3.7015841007232666, 3.538759708404541, 3.7152583599090576, 3.8422789573669434, 3.884699821472168, 3.599134922027588, 3.826996326446533, 3.865656852722168, 3.459383487701416, 3.9870340824127197, 3.843275547027588, 3.6549649238586426, 3.66335129737854, 3.633660316467285, 3.992478132247925, 3.583721160888672, 3.856050491333008, 3.8701629638671875, 3.7747550010681152, 3

In [61]:
# trials.delete_all()

## Cleanup

In [106]:
tunnel.close()

In [146]:
instance.stop()

Stopping instance 403437.


403437: running   $0.1456/hr ssh4.vast.ai:13437  625.3↑ 481.4↓  flops:13.698048T
	1X 10.9GB GTX 1080 Ti, 8.0/32X 15.8GB AMD Ryzen Threadripper 1950X 16-Core Processor
	Successfully loaded tensorflow/tensorflow:1.15.0-gpu-py3-jupyter